# Smart Building System Dataset
Analysis & Data transformation

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plot
import glob
import seaborn as sns
import os
import re

In [2]:
base_path = '../../data/SmartBuildingSystem/KETI/413/'

df_co2 = pd.read_csv(os.path.join(base_path, 'co2.csv'), names=['timestamp', 'co2'], sep=',')
df_co2.head()

,timestamp,co2
0,1377299107,488.0
1,1377299108,495.0
2,1377299110,487.0
3,1377299127,496.0
4,1377299130,495.0


In [3]:
df_hum = pd.read_csv(os.path.join(base_path, 'humidity.csv'), names=['timestamp', 'humidity'], sep=',')
df_hum.head()

,timestamp,humidity
0,1377299107,45.34
1,1377299108,45.34
2,1377299126,45.34
3,1377299131,45.31
4,1377299136,45.34


In [ ]:
df_light = pd.read_csv(os.path.join(base_path, 'light.csv'), names=['timestamp', 'light'], sep=',')
print(df_light.head())

df_pir = pd.read_csv(os.path.join(base_path, 'pir.csv'), names=['timestamp', 'pir'], sep=',')
print(df_pir.head())

df_temp = pd.read_csv(os.path.join(base_path, 'temperature.csv'), names=['timestamp', 'temperature'], sep=',')
print(df_temp.head())

### Importing, transform and export data from Smart Building System

In [4]:
# Applying resampling on the given dataframe
def resample_data(dataframe, col_name, timestamp_name):
    dataframe[timestamp_name] = pd.to_datetime(dataframe[timestamp_name], unit='s')

    aggregates = {
        col_name: 'mean'
    }

    return dataframe.resample('60S', on=timestamp_name).agg(aggregates)

# Return from each room directory a complete dataframe of all the separated features/columns
def transform_room_data(path):
    columns = ['co2', 'humidity', 'light', 'pir', 'temperature']
    df = pd.DataFrame()

    for col_name in columns:
        filename = os.path.join(path, col_name + '.csv')

        df_feature = pd.read_csv(filename, names=['timestamp', col_name], sep=',')

        df_sampled = resample_data(df_feature, col_name, 'timestamp')

        if (columns.index(col_name) < 1):
            df = df_sampled
        else:
            df[col_name] = df_sampled[col_name]
    
    df.dropna(subset=columns, inplace=True)

    return df

# Import, transform and export all data from KETI
def import_data(base_path='../../data/SmartBuildingSystem/KETI/*'):
    all_room_dirs = glob.glob(base_path)

    pattern = re.compile('^[0-9]+$')

    all_room_dirs = [f for f in all_room_dirs if pattern.match(f[-3:])]

    for room_dir in all_room_dirs:
        df = transform_room_data(room_dir)

        output_path = '../../data/SmartBuildingSystem/data/' + room_dir[-3:] + '.csv'

        df.to_csv(output_path)

In [6]:
import_data()

In [5]:
df_413 = transform_room_data('../../data/SmartBuildingSystem/KETI/413/')
df_413.head(10)

,co2,humidity,light,pir,temperature
timestamp,,,,,
2013-08-23 23:05:00,494.727273,45.3300,96.555556,0.0,23.927778
2013-08-23 23:06:00,496.666667,45.3275,96.833333,0.0,23.938333
2013-08-23 23:07:00,498.916667,45.3250,97.333333,0.0,23.942500
2013-08-23 23:08:00,504.166667,45.3125,97.083333,0.0,23.952500
2013-08-23 23:09:00,499.500000,45.3100,97.416667,0.0,23.955833
2013-08-23 23:10:00,502.750000,45.3100,97.000000,0.0,23.966667
2013-08-23 23:11:00,498.333333,45.3100,97.083333,0.0,23.970833
2013-08-23 23:12:00,509.090909,45.3100,98.083333,0.0,23.978333
2013-08-23 23:13:00,512.538462,45.3075,98.750000,0.0,23.984167


In [34]:
df_413.dropna(subset=['co2', 'humidity', 'light', 'pir', 'temperature'], inplace=True)

In [37]:
df_413.isna().sum()

co2            0
humidity       0
light          0
pir            0
temperature    0
dtype: int64